Programming Assignment:


Готовим LDA по рецептам

Как вы уже знаете, в тематическом моделировании делается предположение о том, что для определения тематики порядок слов в документе не важен; об этом гласит гипотеза «мешка слов». Сегодня мы будем работать с несколько нестандартной для тематического моделирования коллекцией, которую можно назвать «мешком ингредиентов», потому что на состоит из рецептов блюд разных кухонь. Тематические модели ищут слова, которые часто вместе встречаются в документах, и составляют из них темы. Мы попробуем применить эту идею к рецептам и найти кулинарные «темы». Эта коллекция хороша тем, что не требует предобработки. Кроме того, эта задача достаточно наглядно иллюстрирует принцип работы тематических моделей.

Для выполнения заданий, помимо часто используемых в курсе библиотек, потребуются модули json и gensim. Первый входит в дистрибутив Anaconda, второй можно поставить командой

pip install gensim

Построение модели занимает некоторое время. На ноутбуке с процессором Intel Core i7 и тактовой частотой 2400 МГц на построение одной модели уходит менее 10 минут.


Загрузка данных

Коллекция дана в json-формате: для каждого рецепта известны его id, кухня (cuisine) и список ингредиентов, в него входящих. Загрузить данные можно с помощью модуля json (он входит в дистрибутив Anaconda):

In [1]:
import json

In [2]:
with open("recipes.json") as f:
    recipes = json.load(f)

In [3]:
print recipes[0]

{u'cuisine': u'greek', u'id': 10259, u'ingredients': [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']}


Составление корпуса

In [4]:
from gensim import corpora, models
import numpy as np 

C:\Users\dim\Anaconda2\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


Наша коллекция небольшая, и целиком помещается в оперативную память. Gensim может работать с такими данными и не требует их сохранения на диск в специальном формате. Для этого коллекция должна быть представлена в виде списка списков, каждый внутренний список соответствует отдельному документу и состоит из его слов. Пример коллекции из двух документов:

[["hello", "world"], ["programming", "in", "python"]]

Преобразуем наши данные в такой формат, а затем создадим объекты corpus и dictionary, с которыми будет работать модель.


In [5]:
texts = [recipe["ingredients"] for recipe in recipes]
dictionary = corpora.Dictionary(texts)   # составляем словарь
corpus = [dictionary.doc2bow(text) for text in texts]  

In [6]:
print texts[0]
print corpus[0]

[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']
[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]


У объекта dictionary есть полезная переменная dictionary.token2id, позволяющая находить соответствие между ингредиентами и их индексами.

Обучение модели

Задание 1. Обучите модель LDA с 40 темами, установив количество проходов по коллекции 5 и оставив остальные параметры по умолчанию.

Затем вызовите метод модели show_topics, указав количество тем 40 и количество токенов 10, и сохраните результат (топы ингредиентов в темах) в отдельную переменную. Если при вызове метода show_topics указать параметр formatted=True, то топы ингредиентов будет удобно выводить на печать, если formatted=False, будет удобно работать со списком программно. Выведите топы на печать, рассмотрите темы, а затем ответьте на вопрос:

Сколько раз ингредиенты "salt", "sugar", "water", "mushrooms", "chicken", "eggs" встретились среди топов-10 всех 40 тем? При ответе не нужно учитывать составные ингредиенты, например, "hot water".

Передайте 6 чисел в функцию save_answers1 и загрузите сгенерированный файл в форму.

У gensim нет возможности фиксировать случайное приближение через параметры метода, но библиотека использует numpy для инициализации матриц. Поэтому, по утверждению автора библиотеки, фиксировать случайное приближение нужно командой, которая написана в следующей ячейке. Перед строкой кода с построением модели обязательно вставляйте указанную строку фиксации random.seed.


In [7]:
def save_answers1(c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs):
    with open("cooking_LDA_pa_task1.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [c_salt, c_sugar, c_water, c_mushrooms, c_chicken, c_eggs]]))

In [82]:
%%time
np.random.seed(76543)
model=models.ldamodel.LdaModel(corpus,id2word=dictionary,num_topics=40,passes=5)

Wall time: 5min 17s


In [12]:
topics=model.show_topics(num_topics=40,num_words=10,formatted=False)

In [22]:
words=[]
for topic in topics:
    for word_proba in topic[1]:
        words.append(word_proba[0])

In [29]:
print words.count("salt")
print words.count("sugar")
print words.count("water")
print words.count("mushrooms")
print words.count("chicken")
print words.count("eggs")

23
9
10
0
1
2


In [31]:
save_answers1(23,9,10,0,1,2)

Фильтрация словаря

В топах тем гораздо чаще встречаются первые три рассмотренных ингредиента, чем последние три. При этом наличие в рецепте курицы, яиц и грибов яснее дает понять, что мы будем готовить, чем наличие соли, сахара и воды. Таким образом, даже в рецептах есть слова, часто встречающиеся в текстах и не несущие смысловой нагрузки, и поэтому их не желательно видеть в темах. Наиболее простой прием борьбы с такими фоновыми элементами — фильтрация словаря по частоте. Обычно словарь фильтруют с двух сторон: убирают очень редкие слова (в целях экономии памяти) и очень частые слова (в целях повышения интерпретируемости тем). Мы уберем только частые слова.

In [32]:
import copy
dictionary2 = copy.deepcopy(dictionary)

Задание 2. У объекта dictionary2 есть переменная dfs — это словарь, ключами которого являются id токена, а элементами — число раз, сколько слово встретилось во всей коллекции. Сохраните в отдельный список ингредиенты, которые встретились в коллекции больше 4000 раз. Вызовите метод словаря filter_tokens, подав в качестве первого аргумента полученный список популярных ингредиентов. Вычислите две величины: dict_size_before и dict_size_after — размер словаря до и после фильтрации.

Затем, используя новый словарь, создайте новый корпус документов, corpus2, по аналогии с тем, как это сделано в начале ноутбука. Вычислите две величины: corpus_size_before и corpus_size_after — суммарное количество ингредиентов в корпусе (для каждого документа вычислите число различных ингредиентов в нем и просуммируйте по всем документам) до и после фильтрации.

Передайте величины dict_size_before, dict_size_after, corpus_size_before, corpus_size_after в функцию save_answers2 и загрузите сгенерированный файл в форму.


In [40]:
id_common_words=[]
for key,value in dictionary2.dfs.iteritems():
    if value>4000:
        id_common_words.append(key)

In [43]:
common_words=[dictionary[id_word] for id_word in id_common_words]
print common_words

[u'garlic', u'pepper', u'ground black pepper', u'salt', u'vegetable oil', u'butter', u'water', u'onions', u'sugar', u'olive oil', u'garlic cloves', u'all-purpose flour']


In [44]:
dictionary2.filter_tokens(bad_ids=id_common_words)

In [46]:
dict_size_before=len(dictionary)
dict_size_before

6714

In [47]:
dict_size_after=len(dictionary2)
dict_size_after

6702

In [48]:
corpus2 = [dictionary2.doc2bow(text) for text in texts]  

In [52]:
corpus_size_before=0
for text in corpus:
    corpus_size_before+=len(text)
corpus_size_before

428249

In [53]:
corpus_size_after=0
for text in corpus2:
    corpus_size_after+=len(text)
corpus_size_after

343665

In [54]:
def save_answers2(dict_size_before, dict_size_after, corpus_size_before, corpus_size_after):
    with open("cooking_LDA_pa_task2.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [dict_size_before, dict_size_after, corpus_size_before, corpus_size_after]]))

In [55]:
save_answers2(dict_size_before,dict_size_after,corpus_size_before,corpus_size_after)

Сравнение когерентностей

Задание 3. Постройте еще одну модель по корпусу corpus2 и словарю dictionary2, остальные параметры оставьте такими же, как при первом построении модели. Сохраните новую модель в другую переменную (не перезаписывайте предыдущую модель). Не забудьте про фиксирование seed!

Затем воспользуйтесь методом top_topics модели, чтобы вычислить ее когерентность. Передайте в качестве аргумента соответствующий модели корпус. Метод вернет список кортежей (топ токенов, когерентность), отсортированных по убыванию последней. Вычислите среднюю по всем темам когерентность для каждой из двух моделей и передайте в функцию save_answers3. 

In [83]:
%%time
np.random.seed(76543) 
model2=models.ldamodel.LdaModel(corpus2,id2word=dictionary2,num_topics=40,passes=5)

Wall time: 4min 16s


In [84]:
cogerence1=0.0
topics=model.top_topics(corpus)
for topic in topics:
    cogerence1+=topic[1]
cogerence1=cogerence1/len(topics)
print cogerence1

-5.82355670677


In [85]:
cogerence2=0.0
topics=model2.top_topics(corpus2)
for topic in topics:
    cogerence2+=topic[1]
cogerence2=cogerence2/len(topics)
print cogerence2

-8.30258722652


In [72]:
def save_answers3(coherence, coherence2):
    with open("cooking_LDA_pa_task3.txt", "w") as fout:
        fout.write(" ".join(["%3f"%el for el in [coherence, coherence2]]))

In [89]:
save_answers3(cogerence1,cogerence2)

Считается, что когерентность хорошо соотносится с человеческими оценками интерпретируемости тем. Поэтому на больших текстовых коллекциях когерентность обычно повышается, если убрать фоновую лексику. Однако в нашем случае этого не произошло.

Изучение влияния гиперпараметра alpha

В этом разделе мы будем работать со второй моделью, то есть той, которая построена по сокращенному корпусу.

Пока что мы посмотрели только на матрицу темы-слова, теперь давайте посмотрим на матрицу темы-документы. Выведите темы для нулевого (или любого другого) документа из корпуса, воспользовавшись методом get_document_topics второй модели:


In [93]:
documents=model2.get_document_topics(corpus2[0])
print documents

[(25, 0.128125), (30, 0.13376448), (31, 0.62248558)]


Также выведите содержимое переменной .alpha второй модели:

In [94]:
print model2.alpha

[ 0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025
  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025  0.025]


У вас должно получиться, что документ характеризуется небольшим числом тем. Попробуем поменять гиперпараметр alpha, задающий априорное распределение Дирихле для распределений тем в документах.

Задание 4. Обучите третью модель: используйте сокращенный корпус (corpus2 и dictionary2) и установите параметр alpha=1, passes=5. Не забудьте про фиксацию seed! Выведите темы новой модели для нулевого документа; должно получиться, что распределение над множеством тем практически равномерное. Чтобы убедиться в том, что во второй модели документы описываются гораздо более разреженными распределениями, чем в третьей, посчитайте суммарное количество элементов, превосходящих 0.01, в матрицах темы-документы обеих моделей. Другими словами, запросите темы модели для каждого документа с параметром minimum_probability=0.01 и просуммируйте число элементов в получаемых массивах. Передайте две суммы (сначала для модели с alpha по умолчанию, затем для модели в alpha=1) в функцию save_answers4.

In [95]:
np.random.seed(76543)
# здесь код для построения модели:
# обучение модели
%time ldamodel3 = models.LdaModel(corpus2, id2word=dictionary2, alpha=1, num_topics=40, passes=5)

Wall time: 2min 58s


In [96]:
print(ldamodel3.get_document_topics(corpus2[0]))
words_0 = [dictionary2[num] for num, _ in ldamodel3.get_document_topics(corpus2[0])]
words_0

[(0, 0.021344228), (1, 0.021296024), (2, 0.021276595), (3, 0.021334194), (4, 0.021279758), (5, 0.02137013), (6, 0.021296108), (7, 0.021276882), (8, 0.021442581), (9, 0.021315478), (10, 0.021621423), (11, 0.042230144), (12, 0.021276595), (13, 0.047129534), (14, 0.021535086), (15, 0.02154783), (16, 0.021439249), (17, 0.021807974), (18, 0.021276595), (19, 0.021404764), (20, 0.024699612), (21, 0.0212867), (22, 0.021276595), (23, 0.021282062), (24, 0.021789333), (25, 0.021566221), (26, 0.021500509), (27, 0.021338968), (28, 0.021489821), (29, 0.021318583), (30, 0.042581294), (31, 0.07080204), (32, 0.021406434), (33, 0.021305392), (34, 0.021288039), (35, 0.021336373), (36, 0.021286787), (37, 0.021292245), (38, 0.021278353), (39, 0.045373466)]


[u'black olives',
 u'feta cheese crumbles',
 u'garbanzo beans',
 u'grape tomatoes',
 u'purple onion',
 u'romaine lettuce',
 u'seasoning',
 u'eggs',
 u'green tomatoes',
 u'ground pepper',
 u'milk',
 u'plain flour',
 u'thyme',
 u'tomatoes',
 u'yellow corn meal',
 u'chicken livers',
 u'cooking oil',
 u'garlic powder',
 u'green chilies',
 u'grilled chicken breasts',
 u'mayonaise',
 u'soy sauce',
 u'yellow onion',
 u'wheat',
 u'bay leaf',
 u'black pepper',
 u'boneless chicken skinless thigh',
 u'cayenne pepper',
 u'chili powder',
 u'cornflour',
 u'double cream',
 u'garam masala',
 u'garlic paste',
 u'ground cumin',
 u'lemon juice',
 u'natural yogurt',
 u'oil',
 u'passata',
 u'shallots',
 u'baking powder']

In [98]:
count_model2 = 0
count_model3 = 0
for doc in corpus2:
    count_model2 += len(model2.get_document_topics(doc, minimum_probability=0.01))
    count_model3 += len(ldamodel3.get_document_topics(doc, minimum_probability=0.01))
print('Number of elements with probability higher than 0.2')
print('Model 2 (alpha="symmetric"): %d' % count_model2)
print('Model 3 (alpha=1): %d' % count_model3)

Number of elements with probability higher than 0.2
Model 2 (alpha="symmetric"): 198528
Model 3 (alpha=1): 1590960


In [99]:
def save_answers4(count_model2, count_model3):
    with open("cooking_LDA_pa_task4.txt", "w") as fout:
        fout.write(" ".join([str(el) for el in [count_model2, count_model3]]))

In [100]:
save_answers4(count_model2, count_model3)

LDA как способ понижения размерности

Иногда, распределения над темами, найденные с помощью LDA, добавляют в матрицу объекты-признаки как дополнительные, семантические, признаки, и это может улучшить качество решения задачи. Для простоты давайте просто обучим классификатор рецептов на кухни на признаках, полученных из LDA, и измерим точность (accuracy).

Задание 5. Используйте модель, построенную по сокращенной выборке с alpha по умолчанию (вторую модель). Составьте матрицу Θ=p(t|d)Θ=p(t|d) вероятностей тем в документах; вы можете использовать тот же метод get_document_topics, а также вектор правильных ответов y (в том же порядке, в котором рецепты идут в переменной recipes). Создайте объект RandomForestClassifier со 100 деревьями, с помощью функции cross_val_score вычислите среднюю accuracy по трем фолдам (перемешивать данные не нужно) и передайте в функцию save_answers5.

In [104]:
from sklearn.cross_validation import cross_val_score
from sklearn.ensemble import RandomForestClassifier
print len(corpus2), len(recipes)

39774 39774


In [103]:
y = [r["cuisine"] for r in recipes]
X = np.zeros((len(recipes), 40))
for i in range(len(recipes)):
    topics = model2.get_document_topics(corpus2[i])
    for topic in topics:
        idx = topic[0]
        val = topic[1]
        X[i,idx]=val

In [105]:
rf=RandomForestClassifier(n_estimators=100)
scores=cross_val_score(rf,X,y,cv=3)
score=scores.mean()
print score

0.553679157294


In [106]:
def save_answers5(accuracy):
     with open("cooking_LDA_pa_task5.txt", "w") as fout:
        fout.write(str(accuracy))

In [107]:
save_answers5(score)

Для такого большого количества классов это неплохая точность. Вы можете попроовать обучать RandomForest на исходной матрице частот слов, имеющей значительно большую размерность, и увидеть, что accuracy увеличивается на 10–15%. Таким образом, LDA собрал не всю, но достаточно большую часть информации из выборки, в матрице низкого ранга.
LDA — вероятностная модель

Матричное разложение, использующееся в LDA, интерпретируется как следующий процесс генерации документов.

Для документа dd длины ndnd:

    Из априорного распределения Дирихле с параметром alpha сгенерировать распределение над множеством тем: θd∼Dirichlet(α)θd∼Dirichlet(α)
    Для каждого слова w=1,…,ndw=1,…,nd:
        Сгенерировать тему из дискретного распределения t∼θdt∼θd
        Сгенерировать слово из дискретного распределения w∼ϕtw∼ϕt.

Подробнее об этом в Википедии.

В контексте нашей задачи получается, что, используя данный генеративный процесс, можно создавать новые рецепты. Вы можете передать в функцию модель и число ингредиентов и сгенерировать рецепт :)
